# Aprendizaje one-class
Este archivo contiene el preprocesamiento y entrenamiento especifico para los modelos one-class.

****Importanción de librerias y lectura de los datos****

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from category_encoders import TargetEncoder
from sklearn.ensemble import IsolationForest 
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from sklearn.metrics import (
    precision_recall_curve, f1_score, accuracy_score,
    precision_score, recall_score, average_precision_score,
    roc_auc_score, confusion_matrix
)
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.neighbors import LocalOutlierFactor 
from sklearn.feature_selection import mutual_info_classif
from lightgbm import LGBMClassifier
from sklearn.feature_selection import RFECV 

In [ ]:
X_train = pd.read_csv('/kaggle/input/preprocessedbasedadatos/X_train.csv')
y_train = pd.read_csv('/kaggle/input/preprocessedbasedadatos/y_train.csv')
X_val = pd.read_csv('/kaggle/input/preprocessedbasedadatos/X_val.csv')
y_val = pd.read_csv('/kaggle/input/preprocessedbasedadatos/y_val.csv')

In [ ]:
print("Nulos en X_tr_normal_train:", X_train.isna().sum().sum())
print("Nulos por columna:\n", X_train.isna().sum())

In [ ]:
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_val   = X_val.reset_index(drop=True)
y_val   = y_val.reset_index(drop=True)
y_train = y_train.iloc[:, 0]

In [ ]:
#Aplicamos filtrado para el entrenamiento con transacciones normales exclusivamente.    
mask_normal = (y_train == 0)
X_tr_normal_train = X_train.loc[mask_normal].copy()
y_tr_normal_train = y_train.loc[mask_normal].copy()


In [ ]:
print("Nulos en X_tr_normal_train:", X_tr_normal_train.isna().sum().sum())
print("Nulos por columna:\n", X_tr_normal_train.isna().sum())

****Pruebas con diferentes encodings****

In [ ]:
###PRUEBA ENCODING CON ONE HOT ENCODING
#X_tr_normal_train[cat_cols] = X_tr_normal_train[cat_cols].fillna("Unknown")
#X_val[cat_cols]   = X_val[cat_cols].fillna("Unknown")

#encoder = OneHotEncoder(handle_unknown="ignore", sparse=False)
#encoder.fit(X_tr_normal_train[cat_cols])

#X_tr_normal_train_cat = encoder.transform(X_tr_normal_train[cat_cols])
#X_val_cat   = encoder.transform(X_val[cat_cols])
#cat_feature_names = encoder.get_feature_names_out(cat_cols)
#X_tr_normal_train_cat = pd.DataFrame(X_tr_normal_train_cat, columns=cat_feature_names, index=X_tr_normal_train.index)
#X_val_cat   = pd.DataFrame(X_val_cat, columns=cat_feature_names, index=X_val.index)
#X_tr_normal_train_final = pd.concat([X_tr_normal_train.drop(columns=cat_cols).reset_index(drop=True), X_tr_normal_train_cat.reset_index(drop=True)], axis=1)
#X_val_final   = pd.concat([X_val.drop(columns=cat_cols).reset_index(drop=True), X_val_cat.reset_index(drop=True)], axis=1)

#X_tr_normal_train = X_tr_normal_train_final
#X_val   = X_val_final

#print("\nNuevas dimensiones:")
#print("Train:", X_tr_normal_train.shape)
#print("Val:", X_val.shape)

#print("\nNaNs en train:", X_tr_normal_train.isna().sum().sum())
#print("NaNs en val:", X_val.isna().sum().sum())


In [ ]:
#from category_encoders import HashingEncoder

### PRUEBA ENCODING CON HASHING ENCODER
# Definir columnas categóricas
#cat_cols = X_tr_normal_train.select_dtypes(include=['object', 'category']).columns.tolist()

# Definir el encoder (puedes ajustar n_components según dimensionalidad)
#encoder = HashingEncoder(cols=cat_cols, n_components=8)  # prueba con 8-16, depende de cardinalidad

# Ajustar el encoder y transformar
#X_tr_normal_train_cat = encoder.fit_transform(X_tr_normal_train[cat_cols])
#X_val_cat   = encoder.transform(X_val[cat_cols])

# Reconstruir datasets con columnas numéricas originales + categóricas codificadas
#X_tr_normal_train_final = pd.concat(
 #   [X_tr_normal_train.drop(columns=cat_cols, errors='ignore'), X_tr_normal_train_cat],
  #  axis=1
#)
#X_val_final = pd.concat(
 #   [X_val.drop(columns=cat_cols, errors='ignore'), X_val_cat],
 #   axis=1
#)

# Sobrescribir variables
#X_tr_normal_train = X_tr_normal_train_final
#X_val = X_val_final

#print("\nNuevas dimensiones:")
#print("Train:", X_tr_normal_train.shape)
#print("Val:", X_val.shape)

#print("\nNaNs en train:", X_tr_normal_train.isna().sum().sum())
#print("NaNs en val:", X_val.isna().sum().sum())


In [ ]:
###PRUEBA ENCODING CON TARGET ENCODING
#encoder = TargetEncoder(cols=cat_cols, handle_missing='value', handle_unknown='value')
#encoder.fit(X_tr_normal_train[cat_cols], y_normal_train)
#X_tr_normal_train_cat = encoder.transform(X_tr_normal_train[cat_cols])
#X_val_cat   = encoder.transform(X_val[cat_cols])
#X_tr_normal_train_final = pd.concat([X_tr_normal_train.drop(columns=cat_cols, errors='ignore'), X_tr_normal_train_cat], axis=1)
#X_val_final   = pd.concat([X_val.drop(columns=cat_cols, errors='ignore'), X_val_cat], axis=1)
#X_tr_normal_train = X_tr_normal_train_final
#X_val = X_val_final

#print("\nNuevas dimensiones:")
#print("Train:", X_tr_normal_train.shape)
#print("Val:", X_val.shape)

#print("\nNaNs en train:", X_tr_normal_train.isna().sum().sum())
#print("NaNs en val:", X_val.isna().sum().sum())

In [ ]:
###PRUEBA CON FRECUENCY ENCODING 
#cat_cols = X_tr_normal_train.select_dtypes(include=['object', 'category']).columns.tolist()

#for col in cat_cols:
 #   freq = X_tr_normal_train[col].value_counts(normalize=True)
 #   X_tr_normal_train[col] = X_tr_normal_train[col].map(freq)
 #   X_val[col] = X_val[col].map(freq)
 #   X_tr_normal_train[col] = X_tr_normal_train[col].fillna(0)
 #   X_val[col] = X_val[col].fillna(0)

#print("\nNuevas dimensiones:")
#print("Train:", X_tr_normal_train.shape)
#print("Val:", X_val.shape)
#print("\nNaNs en train:", X_tr_normal_train.isna().sum().sum())
#print("NaNs en val:", X_val.isna().sum().sum())

In [ ]:

# Identificar columnas categóricas
cat_cols = X_tr_normal_train.select_dtypes(include=['object', 'category']).columns.tolist()
print(cat_cols)


for col in cat_cols:
    le = LabelEncoder()
    X_tr_normal_train.loc[:, col] = le.fit_transform(X_tr_normal_train[col].astype(str))
    
    if col in X_val.columns:
        vals = X_val[col].astype(str)
        vals = vals.map(lambda x: x if x in le.classes_ else None)
        vals = vals.fillna("UNK")
        if "UNK" not in le.classes_:
            le.classes_ = np.append(le.classes_, "UNK")
        X_val.loc[:, col] = le.transform(vals)

print("\nNuevas dimensiones:")
print("Train:", X_tr_normal_train.shape)
print("Val:", X_val.shape)

In [ ]:
print("Nulos en X_tr_normal_train:", X_tr_normal_train.isna().sum().sum())
print("Nulos por columna:\n", X_tr_normal_train.isna().sum())


****Pruebas de selección de columnas con el top 100**** 

In [ ]:
#discrete_mask = X_tr_normal_train.columns.isin(cat_cols)
#importancias = mutual_info_classif(X_tr_normal_train,y_normal_train,discrete_features=discrete_mask, random_state=42)
#importancias = pd.Series(importancias, index = X_tr_normal_train.columns).sort_values(ascending=False)
#top = 200
#top_columnas = importancias.head(top).index.tolist()

#X_tr_normal_train = X_tr_normal_train[top_columnas]
#X_val = X_val[top_columnas]

#plt.figure(figsize=(10,6))
#importancias.head(70).plot(kind='bar')
#plt.title('Top 100 columnas según relevancia')
#plt.tight_layout()
#plt.show()

In [ ]:

#modelo_importancias = LGBMClassifier(n_estimators=100,random_state=42,n_jobs=-1)
#rfecv= RFECV (estimator=modelo_importancias, step=1,cv=StratifiedKFold(3),scoring= 'average_precision', min_features_to_select=100)
#rfecv.fit(X_tr_normal_train,y_normal_train)
#print("Número óptimo de variables:", rfecv.n_features_)
#selected_features= X_tr_normal_train.columns[rfecv.support_]

#X_tr_normal_train= X_tr_normal_train[selected_features]
#X_val = X_val[selected_features]

#plt.figure(figsize=(8,4))
#plt.plot(
 #   range(1, len(rfecv.cv_results_['mean_test_score']) + 1),
 #   rfecv.cv_results_['mean_test_score']
#)
#plt.xlabel("Número de variables")
#plt.ylabel("PR AUC")
#plt.show()

****Escalado de la base de datos y entrenamiento del modelo LOF****

In [ ]:

#Realizamos el escalado, mejora el rendimiento general de los modelos.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_tr_normal_train)  # solo normales
X_val_scaled = scaler.transform(X_val)

print("Nulos en X_tr_normal_train:", X_tr_normal_train.isna().sum().sum())
print("Nulos por columna:\n", X_tr_normal_train.isna().sum())

#X_tr_normal_train = X_tr_normal_train.reset_index(drop=True)
#y_normal_train = y_normal_train.reset_index(drop=True)

lof = LocalOutlierFactor(n_neighbors=15, contamination=0.035,novelty=True,leaf_size=50, n_jobs =-1)
lof.fit(X_train_scaled)

# Scores en validación
val_scores = -lof.decision_function(X_val_scaled)

# Buscar mejor threshold
precision, recall, thresholds = precision_recall_curve(y_val, val_scores)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-6)
best_idx = np.argmax(f1_scores)
best_thr = thresholds[best_idx]

y_pred = (val_scores >= best_thr).astype(int)

print(f"Mejor F1={f1_scores[best_idx]:.4f}, con threshold={best_thr:.4f}")
print("Accuracy:", accuracy_score(y_val, y_pred))
print("Precision:", precision_score(y_val, y_pred, zero_division=0))
print("Recall:", recall_score(y_val, y_pred, zero_division=0))
print("F1:", f1_score(y_val, y_pred, zero_division=0))
print("PR AUC:", average_precision_score(y_val, val_scores))
print("ROC AUC:", roc_auc_score(y_val, val_scores))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred))


****Entrenamiento del modelo Autoencoder****

In [ ]:
#Definimos las capas del autoencoder

inp = layers.Input(shape=(X_train_scaled.shape[1],))
encoded = layers.Dense(32, activation="relu",
                       activity_regularizer=regularizers.l1(1e-5))(inp)
z = layers.Dense(8, activation="relu")(encoded)
decoded = layers.Dense(32, activation="relu")(z)
out = layers.Dense(X_train_scaled.shape[1], activation="linear")(decoded)

autoencoder = models.Model(inp, out)
encoder = models.Model(inp, z)
autoencoder.compile(optimizer="adam", loss="mae")

early_stop = EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True
)

autoencoder.fit(
    X_train_scaled, X_train_scaled,
    validation_data=(X_val_scaled, X_val_scaled),
    epochs=50, batch_size=16,
    callbacks=[early_stop], verbose=1,
)

recon_val = autoencoder.predict(X_val_scaled)
val_scores = np.mean((X_val_scaled - recon_val) ** 2, axis=1)

# Buscar mejor threshold por F1
precision, recall, thresholds = precision_recall_curve(y_val, val_scores)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-6)
best_idx = np.argmax(f1_scores)
best_thr = thresholds[best_idx]

y_pred = (val_scores >= best_thr).astype(int)

print("\n### Autoencoder ###")
print(f"Mejor F1={f1_scores[best_idx]:.4f}, con threshold={best_thr:.4f}")
print("Accuracy:", accuracy_score(y_val, y_pred))
print("Precision:", precision_score(y_val, y_pred, zero_division=0))
print("Recall:", recall_score(y_val, y_pred, zero_division=0))
print("F1:", f1_score(y_val, y_pred, zero_division=0))
print("PR AUC:", average_precision_score(y_val, val_scores))
print("ROC AUC:", roc_auc_score(y_val, val_scores))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred))



****Entrenamiento del modelo Isolation Forest****

In [ ]:
iso = IsolationForest(n_estimators = 150, max_samples = 512, contamination=0.035,bootstrap= True, random_state=42, n_jobs=-1)
iso.fit(X_train_scaled)
val_scores = -iso.decision_function(X_val_scaled)  # mayor = más anómalo

# Buscar mejor threshold
precision, recall, thresholds = precision_recall_curve(y_val, val_scores)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-6)
best_idx = np.argmax(f1_scores)
best_thr = thresholds[best_idx]

y_pred = (val_scores >= best_thr).astype(int)

print(f"Mejor F1={f1_scores[best_idx]:.4f}, con threshold={best_thr:.4f}")
print("Accuracy:", accuracy_score(y_val, y_pred))
print("Precision:", precision_score(y_val, y_pred, zero_division=0))
print("Recall:", recall_score(y_val, y_pred, zero_division=0))
print("F1:", f1_score(y_val, y_pred, zero_division=0))
print("PR AUC:", average_precision_score(y_val, val_scores))
print("ROC AUC:", roc_auc_score(y_val, val_scores))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred))